In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from IPython.display import HTML, display
import 
#2 Gammas Fix later
#from scipy.stats import gamma
#from scipy.stats.distributions import gamma
#from sklearn.model_selection import GridSearchCV
#from sklearn.neighbors import KernelDensity
#from scipy.stats import kde
#from scipy.special import psi, polygamma
#from scipy.optimize import newton

In [ ]:
def outlierDetection(truck_df):
    